# Setup

https://towardsdatascience.com/10-minutes-to-building-a-cnn-binary-image-classifier-in-tensorflow-4e216b2034aa

In [3]:
import os

In [4]:
train_boats_dir = os.path.join('/home/sara/Desktop/Master-thesis/master_thesis/Dataset/binary/train/boat') 
validation_boats_dir = os.path.join('/home/sara/Desktop/Master-thesis/master_thesis/Dataset/binary/val/boat') 

train_no_boats_dir = os.path.join('/home/sara/Desktop/Master-thesis/master_thesis/Dataset/binary/train/no_boat') 
validation_no_boats_dir = os.path.join('/home/sara/Desktop/Master-thesis/master_thesis/Dataset/binary/val/no_boat') 

In [5]:
train_boats_names = os.listdir(train_boats_dir)
print(train_boats_names[:10])
val_boats_names = os.listdir(validation_boats_dir)
print(val_boats_names[:10])

train_no_boats_names = os.listdir(train_no_boats_dir)
print(train_boats_names[:10])
val_no_boats_names = os.listdir(validation_no_boats_dir)
print(val_boats_names[:10])

['fc23a91b3.jpg', 'fb748313b.jpg', 'fb2b9197a.jpg', 'fbc078158.jpg', 'fbc6cd327.jpg', 'fbb8dcf19.jpg', 'fc1dbfee4.jpg', 'f9d4b9343.jpg', 'fbaa8bf5a.jpg', 'f8d0b6652.jpg']
['f97e961f2.jpg', 'f90c9a01c.jpg', 'f9109d4cf.jpg', 'f98d30216.jpg', 'fa0f86121.jpg', 'f910ee612.jpg', 'f95e90324.jpg', 'f891e184c.jpg', 'f9fb6b59c.jpg', 'fa4a030f2.jpg']
['fc23a91b3.jpg', 'fb748313b.jpg', 'fb2b9197a.jpg', 'fbc078158.jpg', 'fbc6cd327.jpg', 'fbb8dcf19.jpg', 'fc1dbfee4.jpg', 'f9d4b9343.jpg', 'fbaa8bf5a.jpg', 'f8d0b6652.jpg']
['f97e961f2.jpg', 'f90c9a01c.jpg', 'f9109d4cf.jpg', 'f98d30216.jpg', 'fa0f86121.jpg', 'f910ee612.jpg', 'f95e90324.jpg', 'f891e184c.jpg', 'f9fb6b59c.jpg', 'fa4a030f2.jpg']


In [6]:
print('total training boats images:', len(os.listdir(train_boats_dir)))
print('total validation boats images:', len(os.listdir(validation_boats_dir)))

print('total training no boats images:', len(os.listdir(train_no_boats_dir)))
print('total validation no boats images:', len(os.listdir(validation_no_boats_dir)))

total training boats images: 491
total validation boats images: 153
total training no boats images: 1753
total validation no boats images: 589


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        "/home/sara/Desktop/Master-thesis/master_thesis/Dataset/binary/train/",  # This is the source directory for training images
        classes = ['boat', 'no_boat'],
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=120,
        # Use binary labels
        class_mode='binary')

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        "/home/sara/Desktop/Master-thesis/master_thesis/Dataset/binary/val/",  # This is the source directory for training images
        classes = ['boat', 'no_boat'],
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=19,
        # Use binary labels
        class_mode='binary',
        shuffle=True)

Found 2244 images belonging to 2 classes.
Found 742 images belonging to 2 classes.


In [8]:
import tensorflow as tf
import numpy as np
from itertools import cycle


from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score


In [17]:
def create_model():
    return tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (200,200,3)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])
model = create_model()

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 120000)            0         
                                                                 
 dense (Dense)               (None, 128)               15360128  
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 15,360,257
Trainable params: 15,360,257
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [15]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/home/sara/Desktop/Master-thesis/master_thesis/Binary/saves/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [16]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True



history = model.fit(train_generator,
      steps_per_epoch=8,  
      epochs=100,
      verbose=1,
      batch_size=batch_size,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/100
8/8 [==============================] - 9s 1s/step - loss: 0.5236 - accuracy: 0.7979 - val_loss: 0.5591 - val_accuracy: 0.7763
Epoch 2/100
8/8 [==============================] - 8s 1s/step - loss: 0.3980 - accuracy: 0.8312 - val_loss: 0.5215 - val_accuracy: 0.7566
Epoch 3/100
8/8 [==============================] - 9s 1s/step - loss: 0.4320 - accuracy: 0.8010 - val_loss: 0.7001 - val_accuracy: 0.6974
Epoch 4/100
8/8 [==============================] - 9s 1s/step - loss: 0.4247 - accuracy: 0.8083 - val_loss: 1.3022 - val_accuracy: 0.7434
Epoch 5/100
8/8 [==============================] - 9s 1s/step - loss: 0.7670 - accuracy: 0.7740 - val_loss: 1.9350 - val_accuracy: 0.3289
Epoch 6/100
8/8 [==============================] - 9s 1s/step - loss: 1.2480 - accuracy: 0.6781 - val_loss: 1.0063 - val_accuracy: 0.8158
Epoch 7/100
8/8 [==============================] - 9s 1s/step - loss: 0.6955 - accuracy: 0.7437 - val_loss: 0.5743 - val_accuracy: 0.8553
Epoch 8/100
5/8 [=================

KeyboardInterrupt: 

In [18]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
import os
import cv2 as cv


arr = os.listdir("/home/sara/Desktop/Master-thesis/master_thesis/Dataset/images")

'''
# predicting images
fn = arr[7]
path = os.path.join("/home/sara/Desktop/Master-thesis/master_thesis/Dataset/images", fn)
img = image.load_img(path, target_size=(200, 200))
x = image.img_to_array(img)
plt.imshow(x/255.)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]<0.5:
  print(fn + " have a boat")
else:
  print(fn + " does not have a boat")
'''

for fn in arr:
    path = os.path.join("/home/sara/Desktop/Master-thesis/master_thesis/Dataset/images", fn)
    im = image.load_img(path, target_size=(200, 200))
    x_show = image.img_to_array(im)

    x = np.expand_dims(x_show, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)

    print(classes[0])
    if classes[0]<0.5:
      print(fn + " have a boat")
    else:
      print(fn + " does not have a boat")

    cv.imshow('display', x_show/255.)
    cv.waitKey(0)

cv.destroyAllWindows()

[1.]
fdd2ef51e.jpg does not have a boat
[0.]
fdc773c96.jpg have a boat
[0.]
fde26a6e0.jpg have a boat
[0.]
fdca53dc2.jpg have a boat
[1.]
fdcb83231.jpg does not have a boat
[1.]
fdcbcc704.jpg does not have a boat
[1.]
fdd1be404.jpg does not have a boat
[0.]
fdc3d0c7e.jpg have a boat
[1.]
fdbf2b2ac.jpg does not have a boat
[0.]
fddefaa7b.jpg have a boat
[0.]
fdc4b0c62.jpg have a boat
[1.]
fdd4706f9.jpg does not have a boat
[0.]
fdc913c2b.jpg have a boat
[0.]
fdd0268c0.jpg have a boat
[0.]
fdd866b18.jpg have a boat
[1.]
fde1ef107.jpg does not have a boat
